## WEM project: JVC Review Bombing Detection

#### Project's members:  Campos Carvalho Cédric, Feuillade Florian, Ramosaj Nicolas

#### Import Libraires

In [1]:
import pandas as pd
import numpy as np
from review_bombing import *
from tools.read import get_data

%load_ext autoreload
%autoreload 2

#### Load Dataset

In [2]:
dataset = get_data('data/dataset500.csv')
data = [{'game':dataset['name'][i], 'platform':dataset['platform'][i], 
        'grade':comment['grade'][0],'comment':comment['comment'][0],'username':comment['username'][0]}
        for i, comments in enumerate(dataset['comments']) for comment in comments]
df = pd.DataFrame(data, columns=['game', 'platform', 'comment', 'username', 'grade'])
df.head()
#print(len(df))

,game,platform,comment,username,grade
0,Elden Ring,PlayStation 5,"Comme Socrate disait:""les personnes qui font s...",Platon_Socrate,20
1,Elden Ring,PlayStation 5,La quintessence de la formule des Souls.Miyaza...,Chichariito,19
2,Elden Ring,PlayStation 5,Le jeu est excellent et tourne bien sur PS5 en...,TheRealBigBoss,20
3,Elden Ring,PlayStation 5,"Je suis là pour corriger ma note, j'ai retiré ...",keanu77fr,18
4,Elden Ring,PlayStation 5,Tout à fait novice dans les jeux Fromsoftware ...,enguerrand_92,19


#### Data normalization

In [3]:
df['comment_normalized'] = [normalize_lemm_stem(comment) for comment in df['comment']]

#### Statistical Analysis

In [4]:
df_grade_pos = df[df['grade'] > 18]
print("Number of comments with grade 19 or 20:", len(df_grade_pos))
df_grade_neg = df[df['grade'] < 2]
print("Number of comments with grade 0 or 1:", len(df_grade_neg))

Number of comments with grade 19 or 20: 2705
Number of comments with grade 0 or 1: 1168


In [5]:
names_pos = extreme_behaviour(dataframe=df, sentiment='positive')
names_neg = extreme_behaviour(dataframe=df, sentiment='negative')
print("Number of user with a positive behaviour:", len(names_pos))
print("Number of user with a negative behaviour:", len(names_neg))

Number of user with a positive behaviour: 246
Number of user with a negative behaviour: 249


#### Prepare Review Bombing Words and Extract Malicious Comments

In [6]:
print(get_bombing_words())

['0', '0/20', '20', '20/20', 'augmente', 'augmenté', 'augmenter', 'augmentes', 'augmentés', 'baisse', 'baissé', 'baisser', 'baisses', 'baissés', 'bombing', 'boycott', 'boycotts', 'bug', 'bugs', 'colere', 'coleres', 'colère', 'colères', 'contre', 'contré', 'contrer', 'contres', 'contrés', 'descendre', 'descendu', 'descendus', 'ensemble', 'ensembles', 'equilibre', 'equilibrer', 'equilibres', 'equilibré', 'equilibrés', 'équilibre', 'équilibrer', 'équilibres', 'équilibré', 'équilibrés', 'evaluation', 'évaluation', 'evalue', 'evalué', 'évalue', 'évalué', 'evaluer', 'évaluer', 'evalues', 'evalués', 'évalues', 'évalués', 'gars', 'honte', 'honteuse', 'honteuses', 'honteux', 'jeuxvideo', 'jeuxvideocom', 'jvc', 'koi', 'mauvais', 'mauvaise', 'mauvaises', 'mec', 'mecs', 'negatif', 'negatifs', 'negative', 'negatives', 'négatif', 'négatifs', 'négative', 'négatives', 'note', 'noter', 'notes', 'patch', 'patche', 'patcher', 'patché', 'positif', 'positifs', 'positive', 'positives', 'quoi', 'rage', 'ragé

In [7]:
review_bombing_pos = naive_bombing(dataframe=df, sentiment='positive')
review_bombing_neg = naive_bombing(dataframe=df, sentiment='negative')
print(len(review_bombing_pos), len(review_bombing_neg))

1251 712


#### Sentiment compound analysis with Vader

In [8]:
# review_bombing_pos = extract_game_sentiment(dataframe=review_bombing_pos, game='Elden Ring')
review_bombing_pos = extract_game_sentiment(dataframe=review_bombing_pos, game=None)
review_bombing_neg = extract_game_sentiment(dataframe=review_bombing_neg, game=None)

#### Extract only username with extreme behaviour

In [9]:
review_pos_high = review_bombing_pos[review_bombing_pos['username'].isin(names_pos)]
review_neg_high = review_bombing_neg[review_bombing_neg['username'].isin(names_neg)]

#### Predict review bombing comments with high confidence

In [10]:
review_pos_high = predict_review_bombing_table(review_pos_high, sentiment='positive', confidence='High')
review_neg_high = predict_review_bombing_table(review_neg_high, sentiment='negative', confidence='High')

In [11]:
review_pos_high = review_pos_high[['game','comment','username']].set_index('game')
review_pos_high['confidence'] = 'HIGH'
review_neg_high = review_neg_high[['game','comment','username']].set_index('game')
review_neg_high['confidence'] = 'HIGH'

#### Predict review bombing comments with medium confidence

In [12]:
review_pos_medium = predict_review_bombing_table(review_bombing_pos, sentiment='positive', confidence='Medium')
review_neg_medium = predict_review_bombing_table(review_bombing_neg, sentiment='negative', confidence='Medium')

In [13]:
review_pos_medium = review_pos_medium[['game','comment','username']].set_index('game')
review_pos_medium['confidence'] = 'MEDIUM'
review_neg_medium = review_neg_medium[['game','comment','username']].set_index('game')
review_neg_medium['confidence'] = 'MEDIUM'

#### Predict review bombing comments with low confidence

In [14]:
review_pos_low = predict_review_bombing_table(review_bombing_pos, sentiment=None, confidence='Low')
review_neg_low = predict_review_bombing_table(review_bombing_neg, sentiment=None, confidence='Low')

In [15]:
review_pos_low = review_pos_low[['game','comment','username']].set_index('game')
review_pos_low['confidence'] = 'LOW'
review_neg_low = review_neg_low[['game','comment','username']].set_index('game')
review_neg_low['confidence'] = 'LOW'

#### Save results as .csv for visualization

In [16]:
full_review_pos = pd.concat((review_pos_high, review_pos_medium, review_pos_low))
full_review_neg = pd.concat((review_neg_high, review_neg_medium, review_neg_low))
full_review_pos.to_csv('data/positive_bombing.csv', encoding='utf-8-sig')
full_review_neg.to_csv('data/negative_bombing.csv', encoding='utf-8-sig')

#### Example of positive sentiment analysis with Vader

In [17]:
# review_bombing_pos = extract_game_sentiment(dataframe=review_bombing_pos, game='Elden Ring')
review_bombing_pos = extract_game_sentiment(dataframe=review_bombing_pos, game='Gran Turismo 7')
print("The length of the negative bombing is:", len(review_bombing_pos))
review_bombing_pos = review_bombing_pos[review_bombing_pos['username'].isin(names_pos)]
pred = predict_review_bombing(dataframe=review_bombing_pos, sentiment='positive', confidence='High')
print("Comment:", pred[0], "From", pred[1])

The length of the negative bombing is: 27
Comment: 13 sur 20 par la communauté, pourtant on à ici un jeux sérieux solide et qui respecte les passionnés de voitures.
Ok gt7 n'est plus le port- étendard de la simulation de voitures par excellence, comme il l'était a une époque.
C'est pas une révolution graphique ou même niveaux sensation.
Mais honnêtement le jeux fait plus que le taff, un 16 ou 17 c'est pas volé mais voilla comme certains sous côte car le jeux n'est pas pensé comme eux l'on pensé ou par mauvaise foies simple ou se retrouve avec des exclus sony basher. Last of us, horizon, gt7 franchement c'est triste de chier sur un travail de qualité. Jusqu'à quand seigneur, jusqu'à quand. From Kounic89


C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['compound_vader'] = [senti['compound'] for senti in senti_vader]


#### Example of negative sentiment analysis with Vader

In [18]:
# review_bombing_neg = extract_game_sentiment(dataframe=review_bombing_neg, game='Elden Ring')
review_bombing_neg = extract_game_sentiment(dataframe=review_bombing_neg, game='Gran Turismo 7')
print("The length of the negative bombing is:", len(review_bombing_neg))
review_bombing_neg = review_bombing_neg[review_bombing_neg['username'].isin(names_neg)]
pred = predict_review_bombing(dataframe=review_bombing_neg, sentiment='negative', confidence='High')
print("Comment:", pred[0], "From", pred[1])

The length of the negative bombing is: 33
Comment: Bon sinon pourquoi GT7 est un ignominie sans nom?- Online obligatoire pour jouer SOLO alors que Sony s'etait spécifiquement moqué du online obligatoire chez Microsoft dans un tweet officiel datant d'il y a quelques années.- Publicité MENSONGERE (et là les amis on est carrément dans l'illégal)
Description du jeu dans le store : "vous pouvez vendre vos voitures" alors que c'est officiellement impossible.
D'ailleurs on va revenir sur ce point très vite.- Contenu maigre, surtout compte tenu du fait que GT7 a été conçu comme une tentative de "retour au source" de la saga contrairement à l'opus d'avant qui avait déçu à ce niveau.
Dans ce cas là comment expliquer que débloquer les permis internationaux et S ne débloquent AUCUNE course ni compétition.
Attention je parle pas de circuit ! Des circuits ont en a assez (même si ils comptent en rajouter) mais bien de nouvelles compétitions, avec des catégories et des spécificités !
Pour résumer: AUC

#### Example of complete process with a single command line

In [19]:
review, pred = review_bombing_prediction_process(dataframe=df, confidence='Low', sentiment='negative', game='Elden Ring')
print("Comment:", pred[0], "From", pred[1])

Comment: Factuellement mauvais.
Le studio nous à habitué a beaucoup mieux que cet opus qui est franchement risible et indigne des souls.
Le jeu est mal optis et les bugs sont insoutenable (ma config n'es pas le problème j'ai une 3070 et un i9 suivi de 32Go de Ram  ) From dentisteforum


C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['compound_vader'] = [senti['compound'] for senti in senti_vader]


In [20]:
review, pred = review_bombing_prediction_process(dataframe=df, confidence='High', sentiment='positive', game='Gran Turismo 7')
print("Comment:", pred[0], "From", pred[1])

Comment: 13 sur 20 par la communauté, pourtant on à ici un jeux sérieux solide et qui respecte les passionnés de voitures.
Ok gt7 n'est plus le port- étendard de la simulation de voitures par excellence, comme il l'était a une époque.
C'est pas une révolution graphique ou même niveaux sensation.
Mais honnêtement le jeux fait plus que le taff, un 16 ou 17 c'est pas volé mais voilla comme certains sous côte car le jeux n'est pas pensé comme eux l'on pensé ou par mauvaise foies simple ou se retrouve avec des exclus sony basher. Last of us, horizon, gt7 franchement c'est triste de chier sur un travail de qualité. Jusqu'à quand seigneur, jusqu'à quand. From Kounic89


C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['compound_vader'] = [senti['compound'] for senti in senti_vader]
